
- kd_load for constants that need to be loaded into ram. kd_end_load
- objcopy into biary data. pushsection popsection. local labels https://discuss.systems/@pkhuong/114939754477593015
- kd_always
- Checking memory reads and writes aren't really state functions. Hmm.
- equivalence checking. kd_sync mylabel "(rax + 7)"  kd_sync mylabel "(select ram 17)"  Two expressions that should be equal. kd_sync_all? ghost_channels? kd_send mylabel "expr". kd_recv varname mylabel. Could just enable reading the ghost vars of the other one. And give a fixed composition ordering. Or just tying all labels? kd_sync_cut ?
- bigger more interesting examples. simd?
- Still unclear if hitting Entry should be something like hitting a sealed precondition (whether entry is also a trace killer). I'd need to track which entry can reach which exits to make this work.
- true / false shortcuts
- trace limit, step bounds
- Interactive VC

- 32 bit arch
- Make CLI.
- better countermodel presentation. could always be btter

Memcopy example
Play with devices on pico or arduino or something. esp32?

Build toy compiler to output

Floyd flowcharts https://userweb.cs.txstate.edu/~rp31/papersSQ/Floyd.pdf 
Very interesting. Quite similar to what I've been doing. The labelled flowchart is the proof object? Akin to cyclic proofs. What if we took non well founded set something? Then well foundedness is less of a concern. The tree nature of ordinary proofs is like comitting to structured programming. Maybe we're missing out on something here.

verilog co-verification

https://www.philipzucker.com/assembly_verify/
https://www.philipzucker.com/asm_verify2/

Local labels and pushsection popsection

euclids algo. Sum 1 - n
Knuth stuff?
list reversal. memcpy
s2n examples
frama-c
vst examples

https://vst.cs.princeton.edu/veric/ vst book. interesting

https://xavierleroy.org/CdF/2023-2024/8.pdf 
https://xavierleroy.org/CdF/2020-2021/ program logics
https://github.com/xavierleroy/cdf-program-logics
https://github.com/xavierleroy/cdf-mech-sem

Annotated flochart --> VCs as a semantics. That's an interesting persepctive.

https://github.com/NationalSecurityAgency/ghidra/tree/master/Ghidra/Extensions/SymbolicSummaryZ3/src/main/java/ghidra/pcode/emu/symz3


gries science of programming

typed assembly 
pcc - knuckeldragger proof objects
foundational pcc


# Bigger Examples


wp-manual
hoare model, typed model, bytes model, region model

assigns
foundational pcc

inject helpers?

writeable
readable
tagged memory

## Swap

In [ ]:
%%file /tmp/swap.c

/*@ requires \valid(a) && \valid(b);
@ ensures A: *a == \old(*b) ;
@ ensures B: *b == \old(*a) ;
@ assigns *a,*b ;
@*/
void swap(int *a,int *b);

void swap(int *a,int *b)
{
    int tmp = *a ;
    *a = *b ;
    *b = tmp ;
    return ;
}


Overwriting /tmp/swap.c


In [8]:
! frama-c -wp -wp-verbose 0 -wp-rte /tmp/swap.c -then -report

[kernel] Parsing /tmp/swap.c (with preprocessing)
[rte:annot] annotating function swap
[report] Computing properties status...

--------------------------------------------------------------------------------
--- Properties of Function 'swap'
--------------------------------------------------------------------------------

[  Valid  ] Post-condition 'A'
            by Wp.typed.
[  Valid  ] Post-condition 'B'
            by Wp.typed.
[  Valid  ] Exit-condition (generated)
            by Unreachable Annotations.
[  Valid  ] Termination-condition (generated)
            by Trivial Termination.
[  Valid  ] Assigns (file /tmp/swap.c, line 5)
            by Wp.typed.
[  Valid  ] Assertion 'rte,mem_access' (file /tmp/swap.c, line 11)
            by Wp.typed.
[  Valid  ] Assertion 'rte,mem_access' (file /tmp/swap.c, line 12)
            by Wp.typed.
[  Valid  ] Assertion 'rte,mem_access' (file /tmp/swap.c, line 12)
            by Wp.typed.
[  Valid  ] Assertion 'rte,mem_access' (file /tmp/swap

In [10]:
! gcc -c -O1 -S /tmp/swap.c -o /tmp/swap.s && cat /tmp/swap.s

	.file	"swap.c"
	.text
	.globl	swap
	.type	swap, @function
swap:
.LFB0:
	.cfi_startproc
	endbr64
	movl	(%rdi), %eax
	movl	(%rsi), %edx
	movl	%edx, (%rdi)
	movl	%eax, (%rsi)
	ret
	.cfi_endproc
.LFE0:
	.size	swap, .-swap
	.ident	"GCC: (Ubuntu 13.3.0-6ubuntu2~24.04) 13.3.0"
	.section	.note.GNU-stack,"",@progbits
	.section	.note.gnu.property,"a"
	.align 8
	.long	1f - 0f
	.long	4f - 1f
	.long	5
0:
	.string	"GNU"
1:
	.align 8
	.long	0xc0000002
	.long	3f - 2f
2:
	.long	0x3
3:
	.align 8
4:


Maybe allowing global assumes would be useful for declaring intermediate constants? global asserts also?
This is kind of too nice.

I would also like to see the ghost assignments in the trace


In [ ]:
%%file /tmp/swap2.s
.include "/tmp/knuckle.s"
kd_prelude "(declare-const olda (_ BitVec 32))"
kd_prelude "(declare-const oldb (_ BitVec 32))"
kd_prelude "(declare-const gap (_ BitVec 64))"
#kd_prelude "(define-const mybv16 (_ bv16 64))" # seems to thrash bv16 notation
#kd_prelude "(define-fun has_gap ((x (_ BitVec 64)) (y (_ BitVec 64)) (g (_ BitVec 64))) (or (bvule (bvadd x y) g) "
#kd_prelude "(define-fun bvgap ((x (_ BitVec 64)) (y (_ BitVec 64)) (g (_ BitVec 64))) (distinct x (bvadd x 1) (bvadd x 2) (bvadd x 3) y (bvadd y 1) (bvadd y 2) (bvadd y 3))"

	.globl	swap
	.type	swap, @function
kd_entry swap "(and (= RDI (_ bv16 64)) (bvuge RSI (_ bv32 64)))"
#kd_entry swap "(and (bvule gap (_ (= (bvadd))"
.LFB0:
kd_assign save_olda olda "(select ram32 RDI)"
kd_assign save_oldb oldb "(select ram32 RSI)"
	.cfi_startproc
	endbr64
	movl	(%rdi), %eax
	movl	(%rsi), %edx
	movl	%edx, (%rdi)
	movl	%eax, (%rsi)
kd_exit myexit "(and (= (select ram32 RSI) olda) (= (select ram32 RDI) oldb))"
	ret
	.cfi_endproc


Overwriting /tmp/swap2.s


In [45]:
! python3 -m kdrag.contrib.pcode /tmp/swap2.s

Processing /tmp/swap2.s with language ID x86:LE:64:default using assembler as
Constructing Trace Fragments...
Unexpected SP conflict
Executing SpecStmt: Assign(label='save_olda', addr=4194304, name='olda', expr=ram32[RDI])
Executing SpecStmt: Assign(label='save_oldb', addr=4194304, name='oldb', expr=ram32[RSI])
Executing 0x400000/4: ENDBR64  at (4194304, 0) PCODE IMARK ram[400000:4]
Executing 0x400004/2: MOV EAX,dword ptr [RDI] at (4194308, 0) PCODE IMARK ram[400004:2]
Executing 0x400004/2: MOV EAX,dword ptr [RDI] at (4194308, 1) PCODE unique[de00:4] = *[ram]RDI
Executing 0x400004/2: MOV EAX,dword ptr [RDI] at (4194308, 2) PCODE EAX = unique[de00:4]
Executing 0x400004/2: MOV EAX,dword ptr [RDI] at (4194308, 3) PCODE RAX = zext(EAX)
Executing 0x400006/2: MOV EDX,dword ptr [RSI] at (4194310, 0) PCODE IMARK ram[400006:2]
Executing 0x400006/2: MOV EDX,dword ptr [RSI] at (4194310, 1) PCODE unique[de00:4] = *[ram]RSI
Executing 0x400006/2: MOV EDX,dword ptr [RSI] at (4194310, 2) PCODE EDX = u

In [15]:
! gcc -c /tmp/swap2.s -o /tmp/swap2.o

/tmp/swap2.s: Assembler messages:
/tmp/swap2.s:1: Error: can't open /tmp/knuckle.s for reading: No such file or directory
/tmp/swap2.s:2: Error: no such instruction: `kd_prelude "(declare-const olda (_ BitVec 32))"'
/tmp/swap2.s:3: Error: no such instruction: `kd_prelude "(declare-const oldb (_ BitVec 32))"'
/tmp/swap2.s:7: Error: no such instruction: `kd_entry swap "true"'
/tmp/swap2.s:9: Error: no such instruction: `kd_assign olda "(select ram32 RDI)"'
/tmp/swap2.s:10: Error: no such instruction: `kd_assign oldb "(select ram32 RSI)"'
/tmp/swap2.s:17: Error: no such instruction: `kd_exit myexit "(and (= (select ram32 RSI) olda)) (= (select ram32 RDI) oldb))"'


## copy1


In [50]:
%%file /tmp/copy1.c

void copy1(int *src, int *dst){
    *dst = *src;
    return;
}

Overwriting /tmp/copy1.c


In [51]:
! gcc -c -O1 -S /tmp/copy1.c -o /tmp/copy1.s && cat /tmp/copy1.s

	.file	"copy1.c"
	.text
	.globl	copy1
	.type	copy1, @function
copy1:
.LFB0:
	.cfi_startproc
	endbr64
	movl	(%rdi), %eax
	movl	%eax, (%rsi)
	ret
	.cfi_endproc
.LFE0:
	.size	copy1, .-copy1
	.ident	"GCC: (Ubuntu 13.3.0-6ubuntu2~24.04) 13.3.0"
	.section	.note.GNU-stack,"",@progbits
	.section	.note.gnu.property,"a"
	.align 8
	.long	1f - 0f
	.long	4f - 1f
	.long	5
0:
	.string	"GNU"
1:
	.align 8
	.long	0xc0000002
	.long	3f - 2f
2:
	.long	0x3
3:
	.align 8
4:


In [9]:
%%file /tmp/copy1.s
.include "/tmp/knuckle.s"
	.text
	.globl	copy1
	.type	copy1, @function
# Property fails if you comment this out. RDI and RSI cannot alias.
kd_entry copy1 "(distinct  (bvadd RDI (_ bv1 64)) (bvadd RDI (_ bv2 64)) (bvadd RDI (_ bv3 64)) RSI (bvadd RSI (_ bv1 64)) (bvadd RSI (_ bv2 64)) (bvadd RSI (_ bv3 64)))"

.LFB0:
	.cfi_startproc
	endbr64
	movl	(%rdi), %eax
	movl	%eax, (%rsi)
kd_exit myexit "(= (select ram32 RSI) (select ram32 RDI))"
	ret
	.cfi_endproc

Overwriting /tmp/copy1.s


In [12]:
! python3 -m kdrag.contrib.pcode /tmp/copy1.s

Processing /tmp/copy1.s with language ID x86:LE:64:default using assembler as
Constructing Trace Fragments...
Unexpected SP conflict
Executing 0x400000/4: ENDBR64  at (4194304, 0) PCODE IMARK ram[400000:4]
Executing 0x400004/2: MOV EAX,dword ptr [RDI] at (4194308, 0) PCODE IMARK ram[400004:2]
Executing 0x400004/2: MOV EAX,dword ptr [RDI] at (4194308, 1) PCODE unique[de00:4] = *[ram]RDI
Executing 0x400004/2: MOV EAX,dword ptr [RDI] at (4194308, 2) PCODE EAX = unique[de00:4]
Executing 0x400004/2: MOV EAX,dword ptr [RDI] at (4194308, 3) PCODE RAX = zext(EAX)
Executing 0x400006/2: MOV dword ptr [RSI],EAX at (4194310, 0) PCODE IMARK ram[400006:2]
Executing 0x400006/2: MOV dword ptr [RSI],EAX at (4194310, 1) PCODE unique[6a80:4] = EAX
Executing 0x400006/2: MOV dword ptr [RSI],EAX at (4194310, 2) PCODE *[ram]RSI = unique[6a80:4]
Executing SpecStmt: Exit(myexit, 0x400008, ram32[RSI] == ram32[RDI])
Checking verification conditions...
[❌] Exit(myexit, 0x400008, ram32[RSI] == ram32[RDI])
--------

## Memcopy



In [58]:
%%file /tmp/mymemcpy.c
#include <stddef.h>
void mymemcpy(int *dst, const int *src, size_t n) {
    for (size_t i = 0; i < n; i++) {
        dst[i] = src[i];
    }
    return;
}


Writing /tmp/mymemcpy.c


In [61]:
! gcc -c -O1 -S /tmp/mymemcpy.c -o /tmp/mymemcpy.s && cat /tmp/mymemcpy.s

	.file	"mymemcpy.c"
	.text
	.globl	mymemcpy
	.type	mymemcpy, @function
mymemcpy:
.LFB0:
	.cfi_startproc
	endbr64
	testq	%rdx, %rdx
	je	.L1
	movl	$0, %eax
.L3:
	movl	(%rsi,%rax,4), %ecx
	movl	%ecx, (%rdi,%rax,4)
	addq	$1, %rax
	cmpq	%rax, %rdx
	jne	.L3
.L1:
	ret
	.cfi_endproc
.LFE0:
	.size	mymemcpy, .-mymemcpy
	.ident	"GCC: (Ubuntu 13.3.0-6ubuntu2~24.04) 13.3.0"
	.section	.note.GNU-stack,"",@progbits
	.section	.note.gnu.property,"a"
	.align 8
	.long	1f - 0f
	.long	4f - 1f
	.long	5
0:
	.string	"GNU"
1:
	.align 8
	.long	0xc0000002
	.long	3f - 2f
2:
	.long	0x3
3:
	.align 8
4:


In [ ]:
%%file /tmp/mymemcpy.s
.include "/tmp/knuckle.s"
	.text
	.globl	mymemcpy
	.type	mymemcpy, @function
kd_entry mymemcpy "(and (bvuge RDI (_ bv0 64)) (bvule RDI (_ bv2 64)))" # limit size
.LFB0:
	.cfi_startproc
	endbr64
	testq	%rdx, %rdx
	je	.L1
	movl	$0, %eax
.L3:
kd_cut mycut "(EAX)"
	movl	(%rsi,%rax,4), %ecx
	movl	%ecx, (%rdi,%rax,4)
	addq	$1, %rax
	cmpq	%rax, %rdx
	jne	.L3
.L1:
kd_exit myexit "(forall ((offset (_ BitVec 64)) (=> (select ram (bvadd RDI offset) (bvadd RSI offset)))"
	ret
	.cfi_endproc

In [5]:
from kdrag.all import *
from kdrag.contrib.pcode.asmspec import assemble_and_gen_vcs

ctx, vcs = assemble_and_gen_vcs("/tmp/swap2.s")
vc = vcs[0]
print(str(smt.simplify(vc.vc(ctx).arg(1))))

Unexpected SP conflict


Executing SpecStmt: Assign(label='save_olda', addr=4194304, name='olda', expr=ram32[RDI])
Executing SpecStmt: Assign(label='save_oldb', addr=4194304, name='oldb', expr=ram32[RSI])
Executing 0x400000/4: ENDBR64  at (4194304, 0) PCODE IMARK ram[400000:4]
Executing 0x400004/2: MOV EAX,dword ptr [RDI] at (4194308, 0) PCODE IMARK ram[400004:2]
Executing 0x400004/2: MOV EAX,dword ptr [RDI] at (4194308, 1) PCODE unique[de00:4] = *[ram]RDI
Executing 0x400004/2: MOV EAX,dword ptr [RDI] at (4194308, 2) PCODE EAX = unique[de00:4]
Executing 0x400004/2: MOV EAX,dword ptr [RDI] at (4194308, 3) PCODE RAX = zext(EAX)
Executing 0x400006/2: MOV EDX,dword ptr [RSI] at (4194310, 0) PCODE IMARK ram[400006:2]
Executing 0x400006/2: MOV EDX,dword ptr [RSI] at (4194310, 1) PCODE unique[de00:4] = *[ram]RSI
Executing 0x400006/2: MOV EDX,dword ptr [RSI] at (4194310, 2) PCODE EDX = unique[de00:4]
Executing 0x400006/2: MOV EDX,dword ptr [RSI] at (4194310, 3) PCODE RDX = zext(EDX)
Executing 0x400008/2: MOV dword ptr

Cleanup the verification condition.
If we need to go manual, or debug, this is unacceptable.

kd_alias RAX 
kd_size "RAX" 4   # ?
kd_object RAX 4
kd_separated
kd_type RAX ctype

maybe ram32 is too cute. It is a weird construct. The different pieceso f the ram are aliased in an unpexected way
(select32 ram RDI)
(select16 )
(select8 )


In [ ]:
smt.substitute(vc.vc(ctx).arg(1), (MemState.Const("mem0").mem.ram, smt.Lambda([addr], Select(ram64[addr], addr % 8, addr % 8 + 1)))

In [ ]:
offset = 0
for name, vnode in ctx.vnodes.items():
    if vnode.offset >= offset and len(name) < 4:
        
    offset = vnode.offset + vnode.size

In [ ]:
regfile = smt.FreshConst("regfile", smt.ArraySort(self.bits, self.bits))
ram = smt.FreshConsts( smt.ArraySort(self.bits, self.bits))

## Sumn

In [108]:
%%file /tmp/sumn.c

#include <stdio.h>
#include <stdlib.h>
int sumn(int n) {
    int sum = 0;
    for (int i = 1; i <= n; i++) {
        sum += i;
    }
    return sum;
}

Writing /tmp/sumn.c


In [ ]:
! frama-c -eva /tmp/sumn.c 

/bin/bash: line 1: framac: command not found


In [112]:
! gcc -c -O1 -S -o /tmp/sumn.s /tmp/sumn.c && cat /tmp/sumn.s

	.file	"sumn.c"
	.text
	.globl	sumn
	.type	sumn, @function
sumn:
.LFB39:
	.cfi_startproc
	endbr64
	testl	%edi, %edi
	jle	.L4
	addl	$1, %edi
	movl	$1, %eax
	movl	$0, %edx
.L3:
	addl	%eax, %edx
	addl	$1, %eax
	cmpl	%edi, %eax
	jne	.L3
.L1:
	movl	%edx, %eax
	ret
.L4:
	movl	$0, %edx
	jmp	.L1
	.cfi_endproc
.LFE39:
	.size	sumn, .-sumn
	.ident	"GCC: (Ubuntu 13.3.0-6ubuntu2~24.04) 13.3.0"
	.section	.note.GNU-stack,"",@progbits
	.section	.note.gnu.property,"a"
	.align 8
	.long	1f - 0f
	.long	4f - 1f
	.long	5
0:
	.string	"GNU"
1:
	.align 8
	.long	0xc0000002
	.long	3f - 2f
2:
	.long	0x3
3:
	.align 8
4:


In [104]:
%%file /tmp/sumarray.c
#include <stddef.h>
int sumarray(int *arr, size_t n) {
    int sum = 0;
    for (size_t i = 0; i < n; i++) {
        sum += arr[i];
    }
    return sum;
}


Writing /tmp/sumarray.c


In [ ]:
! frama-c 

In [107]:
! gcc /tmp/sumarray.c -c -fverbose-asm -S -o /tmp/sumarray.s -O1 && cat /tmp/sumarray.s

	.file	"sumarray.c"
# GNU C17 (Ubuntu 13.3.0-6ubuntu2~24.04) version 13.3.0 (x86_64-linux-gnu)
#	compiled by GNU C version 13.3.0, GMP version 6.3.0, MPFR version 4.2.1, MPC version 1.3.1, isl version isl-0.26-GMP

# GGC heuristics: --param ggc-min-expand=100 --param ggc-min-heapsize=131072
# options passed: -mtune=generic -march=x86-64 -O1 -fasynchronous-unwind-tables -fstack-protector-strong -fstack-clash-protection -fcf-protection
	.text
	.globl	sumarray
	.type	sumarray, @function
sumarray:
.LFB0:
	.cfi_startproc
	endbr64	
# /tmp/sumarray.c:4:     for (size_t i = 0; i < n; i++) {
	testq	%rsi, %rsi	# n
	je	.L4	#,
	movq	%rdi, %rax	# arr, ivtmp.6
	leaq	(%rdi,%rsi,4), %rcx	#, _19
# /tmp/sumarray.c:3:     int sum = 0;
	movl	$0, %edx	#, <retval>
.L3:
# /tmp/sumarray.c:5:         sum += arr[i];
	addl	(%rax), %edx	# MEM[(int *)_16], <retval>
# /tmp/sumarray.c:4:     for (size_t i = 0; i < n; i++) {
	addq	$4, %rax	#, ivtmp.6
	cmpq	%rcx, %rax	# _19, ivtmp.6
	jne	.L3	#,
.L1:
# /tmp/sumarray.c:

In [96]:
%%file /tmp/copy.c
#include <stddef.h>
void mycopy(int *dest, const int *src, size_t n) {
    for (size_t i = 0; i < n; i++) {
        dest[i] = src[i];
    }
}

Overwriting /tmp/copy.c


In [103]:
! gcc /tmp/copy.c -c -S -o /tmp/copy.s -O1 && cat /tmp/copy.s

	.file	"copy.c"
	.text
	.globl	mycopy
	.type	mycopy, @function
mycopy:
.LFB0:
	.cfi_startproc
	endbr64
	testq	%rdx, %rdx
	je	.L1
	movl	$0, %eax
.L3:
	movl	(%rsi,%rax,4), %ecx
	movl	%ecx, (%rdi,%rax,4)
	addq	$1, %rax
	cmpq	%rax, %rdx
	jne	.L3
.L1:
	ret
	.cfi_endproc
.LFE0:
	.size	mycopy, .-mycopy
	.ident	"GCC: (Ubuntu 13.3.0-6ubuntu2~24.04) 13.3.0"
	.section	.note.GNU-stack,"",@progbits
	.section	.note.gnu.property,"a"
	.align 8
	.long	1f - 0f
	.long	4f - 1f
	.long	5
0:
	.string	"GNU"
1:
	.align 8
	.long	0xc0000002
	.long	3f - 2f
2:
	.long	0x3
3:
	.align 8
4:


# Better Countermodel
Maybe also print all ghost var
or just add dummy conditions

kd_print "expr"
kd_watch

Maybe suggest that some things may be aliasing. Suspicious looking numbers that are too close, or show up in selects.





In [82]:
%%file /tmp/mov42.s
.include "/tmp/knuckle.s"
.global  _start
kd_entry _start "true"
    movq     $42, 8(%rsp)
kd_exit _start_end "(= (select ram (bvadd RSP (_ bv8 64))) (_ bv43 8))"
    ret

Overwriting /tmp/mov42.s


In [90]:
! python3 -m kdrag.contrib.pcode /tmp/mov42.s

Processing /tmp/mov42.s with language ID x86:LE:64:default using assembler as
Constructing Trace Fragments...
Unexpected SP conflict
Executing 0x400000/9: MOV qword ptr [RSP + 0x8],0x2a at (4194304, 0) PCODE IMARK ram[400000:9]
Executing 0x400000/9: MOV qword ptr [RSP + 0x8],0x2a at (4194304, 1) PCODE unique[4e00:8] = 0x8 + RSP
Executing 0x400000/9: MOV qword ptr [RSP + 0x8],0x2a at (4194304, 2) PCODE unique[6e80:8] = 0x2a
Executing 0x400000/9: MOV qword ptr [RSP + 0x8],0x2a at (4194304, 3) PCODE *[ram]unique[4e00:8] = unique[6e80:8]
Executing SpecStmt: Exit(_start_end, 0x400009, ram[RSP + 8] == 43)
Checking verification conditions...
[❌] Exit(_start_end, 0x400009, ram[RSP + 8] == 43)
---------------------------------------------
Trace:
	 Entry(_start, 0x400000, True)
	 0x400000/9: MOV qword ptr [RSP + 0x8],0x2a
	 Exit(_start_end, 0x400009, ram[RSP + 8] == 43)

Countermodel:
{ram[RSP + 8]: 42, RSP: 18446744073709551608}
---------------------------------------------
❌❌❌❌ Some verificati

## 32 bit

In [16]:
%%file /tmp/mov42.s
.include "/tmp/knuckle.s"
    .text
    .globl  myfunc
kd_entry myfunc "true"
    addi    sp, sp, -4       # make room on the stack
    li      t0, 42           # t0 ← 42
    sw      t0, 0(sp)        # [sp] = 42
kd_exit myfunc_end "(= (select ram32 sp) (_ bv42 32))"
    ret



Overwriting /tmp/mov42.s


In [27]:
! python3 -m kdrag.contrib.pcode --help

Usage: python -m kdrag.contrib.pcode [OPTIONS] FILENAME

  Simple program that greets NAME for a total of COUNT times.

Options:
  --langid TEXT  PCode language ID. Run `python3 -m pypcode --list` for
                 options
  --asm TEXT     Assembler to use. Default is 'as'.
  --help         Show this message and exit.


In [33]:
! python3 -m kdrag.contrib.pcode --langid="RISCV:LE:32:default" --asm /home/philip/Downloads/lowrisc-toolchain-gcc-rv32imcb-x86_64-20250710-1/bin/riscv32-unknown-elf-as /tmp/mov42.s

Processing /tmp/mov42.s with language ID RISCV:LE:32:default using assembler /home/philip/Downloads/lowrisc-toolchain-gcc-rv32imcb-x86_64-20250710-1/bin/riscv32-unknown-elf-as
Constructing Trace Fragments
Executing 0x400000/4: addi sp,sp,-0x4 at (4194304, 0) PCODE IMARK ram[400000:4]
Executing 0x400000/4: addi sp,sp,-0x4 at (4194304, 1) PCODE unique[580:4] = 0xfffffffc
Executing 0x400000/4: addi sp,sp,-0x4 at (4194304, 2) PCODE sp = sp + unique[580:4]
Executing 0x400004/4: li t0,0x2a at (4194308, 0) PCODE IMARK ram[400004:4]
Executing 0x400004/4: li t0,0x2a at (4194308, 1) PCODE unique[580:4] = 0x2a
Executing 0x400004/4: li t0,0x2a at (4194308, 2) PCODE t0 = unique[580:4]
Executing 0x400008/4: sw t0,0x0(sp) at (4194312, 0) PCODE IMARK ram[400008:4]
Executing 0x400008/4: sw t0,0x0(sp) at (4194312, 1) PCODE unique[600:4] = 0x0
Executing 0x400008/4: sw t0,0x0(sp) at (4194312, 2) PCODE unique[4180:4] = sp + unique[600:4]
Executing 0x400008/4: sw t0,0x0(sp) at (4194312, 3) PCODE *[ram]uniqu

In [8]:
from kdrag.contrib.pcode.asmspec import assemble_and_check
assemble_and_check("/tmp/mov42.s", langid="RISCV:LE:32:default", as_bin="riscv32-linux-gnu-as").successes

FileNotFoundError: [Errno 2] No such file or directory: 'riscv32-linux-gnu-as'

In [5]:
import archinfo

info = archinfo.ArchPcode("RISCV:LE:32:default")
info.bits

32